In [3]:
from tigeropen.common.consts import (Language,        # 语言
                                Market,           # 市场
                                BarPeriod,        # k线周期
                                QuoteRight)       # 复权类型
from tigeropen.tiger_open_config import TigerOpenClientConfig
from tigeropen.common.util.signature_utils import read_private_key
from tigeropen.quote.quote_client import QuoteClient
import os

def get_client_config():
    """
    https://quant.itigerup.com/#developer 开发者信息获取
    """
    client_config = TigerOpenClientConfig()
    # 使用绝对路径读取私钥文件
    current_dir = os.path.dirname(os.path.abspath("./tiger-test"))
    private_key_path = os.path.join(current_dir, 'keys', 'private.pem')
    client_config.private_key = read_private_key(private_key_path)
    client_config.tiger_id = '20153826'
    # client_config.account = '20240803144534965'
    client_config.account = '498843'
    client_config.language = Language.zh_CN  #可选，不填默认为英语'
    # client_config.timezone = 'US/Eastern' # 可选时区设置
    return client_config

client_config = get_client_config()

# 随后传入配置参数对象来初始化QuoteClient
quote_client = QuoteClient(client_config)

# 完成初始化后，就可以调用quote_client方法来使用调用QuoteClient对象的get_stock_brief方法来查询股票行情了
# 此处以美国股票为例，关于其他支持的市场及标的类型，请参考文档的基本操作部分。
# 对于使用多台设备调用API的用户，需先调用grab_quote_permission进行行情权限的抢占，详情请见基本操作-行情类-通用-grab_quote_permission方法说明


In [ ]:
import pandas as pd
# permissions = quote_client.get_quote_permission() 
# print(permissions)
# market_status = quote_client.get_market_status(Market.US)
timeline = quote_client.get_timeline(['AAPL'], include_hour_trading=False)

# 将 time 转换为对应时区的日期时间
timeline['cn_date'] = pd.to_datetime(timeline['time'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Shanghai')
timeline['us_date'] = pd.to_datetime(timeline['time'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('US/Eastern')

# 查看最新一条数据
print(timeline.iloc[-1].price)
print(timeline.iloc[-1].volume)
# 按symbol过滤
df = timeline.loc[timeline['symbol'] == 'AAPL']

In [22]:
from datetime import datetime, timedelta
bars = quote_client.get_bars(['AAPL'])
#trade_session 可选参数值
# bars = quote_client.get_bars(['AAPL'],period = BarPeriod.ONE_MINUTE, trade_session=TradingSession.PreMarket)
# bars = quote_client.get_bars(['AAPL'],period = BarPeriod.HALF_HOUR, trade_session=TradingSession.AfterHours)
# bars = quote_client.get_bars(['AAPL'],period = BarPeriod.ONE_MINUTE,trade_session=TradingSession.OverNight)

print(bars.head())
end_date = datetime.now()
start_date = end_date - timedelta(days=65)  # 5年数据

# 获取富时中国A50指数数据
a50_symbol = 'AAPL'  # 富时中国A50指数的代码
print(f"正在获取 {a50_symbol} 的数据...")
a50_data = quote_client.get_bars(
    symbols=[a50_symbol],
    period='day',
    begin_time=int(start_date.timestamp() * 1000),
    end_time=int(end_date.timestamp() * 1000)
)

  symbol           time     open      high       low   close    volume  \
0   AAPL  1715832000000  190.470  191.0950  189.6601  189.84  52845230   
1   AAPL  1715918400000  189.510  190.8100  189.1800  189.87  41282925   
2   AAPL  1716177600000  189.325  191.9199  189.0100  191.04  44361275   
3   AAPL  1716264000000  191.090  192.7300  190.9201  192.35  42309401   
4   AAPL  1716350400000  192.265  192.8231  190.2700  190.90  34648547   

         amount next_page_token  
0  1.005126e+10            None  
1  7.840779e+09            None  
2  8.488261e+09            None  
3  8.134183e+09            None  
4  6.632339e+09            None  
正在获取 AAPL 的数据...
